# N Back Task
## Optimal Behaviour

#### Deterministic Task

The task is deterministic thus programming the best possible behaviour is trivial:
- Store the sequence of information
- Trigger if matches